In [ ]:
!apt-get update #update linux
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #download and install openjdk
!wget -q http://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz #download spark binary (gunzip)
!tar xf spark-3.2.1-bin-hadoop2.7.tgz #extract the spark package
!pip install -q findspark #install the findspark package

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [84.8 kB]
Hit:13 http://ppa.laun

In [ ]:
#set environment variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [ ]:
!ls #check directory

sample_data  spark-3.2.1-bin-hadoop2.7	spark-3.2.1-bin-hadoop2.7.tgz


In [ ]:
!ls sample_data #check directory under sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
# import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 

# File location and type
path = 'sample_data/california_housing_train.csv'
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Import csv. The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(path)

df.head() #first row object

Row(longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0)

In [ ]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [ ]:
df.count()

17000

In [ ]:
df.drop_duplicates().count()

17000

In [ ]:
# create a temporary view
df.createOrReplaceTempView("california_housing_train")

In [ ]:
df_sql = spark.sql("SELECT * FROM california_housing_train LIMIT 12")
df_sql.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

In [ ]:
df_sql.count()

12

In [ ]:
df_sql.take(5) #first 5 row objects

[Row(longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0),
 Row(longitude=-114.47, latitude=34.4, housing_median_age=19.0, total_rooms=7650.0, total_bedrooms=1901.0, population=1129.0, households=463.0, median_income=1.82, median_house_value=80100.0),
 Row(longitude=-114.56, latitude=33.69, housing_median_age=17.0, total_rooms=720.0, total_bedrooms=174.0, population=333.0, households=117.0, median_income=1.6509, median_house_value=85700.0),
 Row(longitude=-114.57, latitude=33.64, housing_median_age=14.0, total_rooms=1501.0, total_bedrooms=337.0, population=515.0, households=226.0, median_income=3.1917, median_house_value=73400.0),
 Row(longitude=-114.57, latitude=33.57, housing_median_age=20.0, total_rooms=1454.0, total_bedrooms=326.0, population=624.0, households=262.0, median_income=1.925, median_house_value=65500.0)]

In [ ]:
type(df.take(5))

list

In [ ]:
df_sql = spark.sql("SELECT longitude, latitude, population, households FROM california_housing_train LIMIT 10")
df_sql.show()

+---------+--------+----------+----------+
|longitude|latitude|population|households|
+---------+--------+----------+----------+
|  -114.31|   34.19|    1015.0|     472.0|
|  -114.47|    34.4|    1129.0|     463.0|
|  -114.56|   33.69|     333.0|     117.0|
|  -114.57|   33.64|     515.0|     226.0|
|  -114.57|   33.57|     624.0|     262.0|
|  -114.58|   33.63|     671.0|     239.0|
|  -114.58|   33.61|    1841.0|     633.0|
|  -114.59|   34.83|     375.0|     158.0|
|  -114.59|   33.61|    3134.0|    1056.0|
|   -114.6|   34.83|     787.0|     271.0|
+---------+--------+----------+----------+



In [ ]:
df_sql

DataFrame[longitude: double, latitude: double, population: double, households: double]

In [ ]:
# use sql aliases
df_sql = spark.sql("SELECT longitude as longi, \
  latitude as lati, population as pop, \
  households as hh FROM california_housing_train LIMIT 10")
df_sql.show()

+-------+-----+------+------+
|  longi| lati|   pop|    hh|
+-------+-----+------+------+
|-114.31|34.19|1015.0| 472.0|
|-114.47| 34.4|1129.0| 463.0|
|-114.56|33.69| 333.0| 117.0|
|-114.57|33.64| 515.0| 226.0|
|-114.57|33.57| 624.0| 262.0|
|-114.58|33.63| 671.0| 239.0|
|-114.58|33.61|1841.0| 633.0|
|-114.59|34.83| 375.0| 158.0|
|-114.59|33.61|3134.0|1056.0|
| -114.6|34.83| 787.0| 271.0|
+-------+-----+------+------+



In [ ]:
#filter by population  >= 10000
df_sql = spark.sql("SELECT * FROM california_housing_train WHERE population  >= 10000")
df_sql.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -117.12|   33.52|               4.0|    30401.0|        4957.0|   13251.0|    4339.0|       4.5841|          212300.0|
|  -117.42|   33.35|              14.0|    25135.0|        4819.0|   35682.0|    4769.0|       2.5729|          134400.0|
|  -117.61|    34.1|               9.0|    18956.0|        4095.0|   10323.0|    3832.0|       3.6033|          132600.0|
|  -117.74|   33.89|               4.0|    37937.0|        5471.0|   16122.0|    5189.0|       7.4947|          366300.0|
|  -117.75|   34.01|               4.0|    22128.0|        3522.0|   10450.0|    3258.0|       6.1287|          289600.0|
|  -117.78|   34.03|    

In [ ]:
#filter by population  >= 10000 and households>=5000
df_sql = spark.sql("SELECT * FROM california_housing_train \
  WHERE population >= 10000 and households>=5000 \
  ORDER BY housing_median_age DESC")
df_sql.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -121.79|   36.64|              11.0|    32627.0|        6445.0|   28566.0|    6082.0|       2.3087|          118800.0|
|  -117.78|   34.03|               8.0|    32054.0|        5290.0|   15507.0|    5050.0|       6.0191|          253900.0|
|  -117.74|   33.89|               4.0|    37937.0|        5471.0|   16122.0|    5189.0|       7.4947|          366300.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+



In [ ]:
#aggregation
df_sql = spark.sql("SELECT round(longitude,0) as longitude, round(latitude,0) as latitude, \
                    min(total_bedrooms) as min_total_bedrooms, round(avg(median_income),2) as avg_median_income, \
                    max(households) as max_households \
                    FROM california_housing_train \
                    WHERE longitude BETWEEN -124 AND -122 \
                    GROUP BY round(longitude,0), round(latitude,0) \
                    ORDER BY round(longitude,0), round(latitude,0)")
df_sql.show()

+---------+--------+------------------+-----------------+--------------+
|longitude|latitude|min_total_bedrooms|avg_median_income|max_households|
+---------+--------+------------------+-----------------+--------------+
|   -124.0|    39.0|             151.0|             2.78|         803.0|
|   -124.0|    40.0|             340.0|             1.78|         486.0|
|   -124.0|    41.0|             238.0|             2.17|         414.0|
|   -124.0|    42.0|             357.0|             3.05|         314.0|
|   -123.0|    38.0|               1.0|             4.34|        2200.0|
|   -123.0|    39.0|              17.0|             2.87|        1088.0|
|   -123.0|    40.0|              28.0|             1.97|         627.0|
|   -123.0|    41.0|              84.0|             2.34|         757.0|
|   -123.0|    42.0|             308.0|             2.28|         869.0|
|   -122.0|    37.0|               4.0|             5.93|        3589.0|
|   -122.0|    38.0|               3.0|            

In [ ]:
#find out duplicated longitude and latitude records
df_sql = spark.sql("SELECT longitude, latitude, COUNT(*) FROM california_housing_train \
  GROUP BY longitude, latitude \
  HAVING COUNT(*) > 1 \
  ORDER BY longitude, latitude \
  ")
df_sql.show()

+---------+--------+--------+
|longitude|latitude|count(1)|
+---------+--------+--------+
|  -124.18|   40.78|       3|
|  -124.17|    40.8|       3|
|  -124.16|   40.78|       3|
|  -124.16|   40.79|       2|
|  -124.16|    40.8|       2|
|  -124.14|   40.59|       2|
|  -124.13|   40.79|       2|
|  -124.11|   40.93|       2|
|   -124.1|    40.5|       2|
|  -124.09|   40.88|       3|
|  -124.07|   40.87|       2|
|  -123.79|   39.44|       3|
|  -123.22|   39.15|       2|
|  -123.21|   39.15|       2|
|  -122.92|   39.05|       2|
|  -122.91|   39.05|       2|
|  -122.82|   38.39|       2|
|  -122.76|   38.44|       2|
|  -122.75|   38.46|       2|
|  -122.74|   38.44|       2|
+---------+--------+--------+
only showing top 20 rows



In [ ]:
# drop the view if it exists
spark.sql("""
  DROP VIEW IF EXISTS longitude_latitude_dup; 
""") 


DataFrame[]

In [ ]:
# create another temporary view
spark.sql("""
  
  CREATE TEMPORARY VIEW longitude_latitude_dup AS 
  SELECT longitude, latitude, COUNT(*) AS record_count FROM california_housing_train 
  GROUP BY longitude, latitude 
  HAVING COUNT(*) > 1 
  ORDER BY longitude, latitude 
""") 


DataFrame[]

In [ ]:
df_dup = spark.sql("SELECT * FROM longitude_latitude_dup") 
df_dup.show()

+---------+--------+--------+
|longitude|latitude|count(1)|
+---------+--------+--------+
|  -124.18|   40.78|       3|
|  -124.17|    40.8|       3|
|  -124.16|   40.78|       3|
|  -124.16|   40.79|       2|
|  -124.16|    40.8|       2|
|  -124.14|   40.59|       2|
|  -124.13|   40.79|       2|
|  -124.11|   40.93|       2|
|   -124.1|    40.5|       2|
|  -124.09|   40.88|       3|
|  -124.07|   40.87|       2|
|  -123.79|   39.44|       3|
|  -123.22|   39.15|       2|
|  -123.21|   39.15|       2|
|  -122.92|   39.05|       2|
|  -122.91|   39.05|       2|
|  -122.82|   38.39|       2|
|  -122.76|   38.44|       2|
|  -122.75|   38.46|       2|
|  -122.74|   38.44|       2|
+---------+--------+--------+
only showing top 20 rows



In [ ]:
# join two views
df_join = spark.sql("""SELECT v1.*, v2.record_count 
                     FROM california_housing_train v1 
                     INNER JOIN longitude_latitude_dup v2 
                     ON v1.longitude = v2.longitude
                     AND v1.latitude = v2.latitude
                     ORDER BY v1.longitude, v1.latitude, housing_median_age
                     """)
df_join.show()   

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|record_count|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------+
|  -124.18|   40.78|              33.0|     1076.0|         222.0|     656.0|     236.0|       2.5096|           72200.0|           3|
|  -124.18|   40.78|              34.0|     1592.0|         364.0|     950.0|     317.0|       2.1607|           67000.0|           3|
|  -124.18|   40.78|              37.0|     1453.0|         293.0|     867.0|     310.0|       2.5536|           70200.0|           3|
|  -124.17|    40.8|              52.0|      661.0|         316.0|     392.0|     244.0|        0.957|           60000.0|           3|
|  -124.17|    40.8|              52.0|     1557.0|    

In [ ]:
# drop_duplicates will only keep the first row for each group of same longitude and latitude
df_join.drop_duplicates(['longitude','latitude']).orderBy(["longitude", "latitude"], ascending=[1, 1]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|record_count|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------+
|  -124.18|   40.78|              33.0|     1076.0|         222.0|     656.0|     236.0|       2.5096|           72200.0|           3|
|  -124.17|    40.8|              52.0|      661.0|         316.0|     392.0|     244.0|        0.957|           60000.0|           3|
|  -124.16|   40.78|              43.0|     2241.0|         446.0|     932.0|     395.0|       2.9038|           82000.0|           3|
|  -124.16|   40.79|              52.0|     2148.0|         421.0|     975.0|     430.0|       2.2566|           92700.0|           2|
|  -124.16|    40.8|              52.0|     2416.0|    